In [102]:
import os
from dotenv import load_dotenv
import requests
from PyPDF2 import PdfReader
load_dotenv(override=True)

from groq import Groq
import gradio as gr



In [103]:
# pushover_user = os.getenv("PUSHOVER_USER")
# pushover_api= os.getenv("PUSHOVER_TOKEN")
# pushover_url = "https://api.pushover.net/1/messages.json"

In [104]:
# if pushover_user:
#     print(f'Pushover user found and starts with {pushover_user[:4]}')
# else:
#     print("Pushover user not found")

# if pushover_api:
#     print(f'Pushover API found and starts with {pushover_api[:4]}')
# else:
#     print("Pushover API not found")

    

In [97]:
def push(text):
    payload = {
        "token": pushover_api,
        "user": pushover_user,
        "message": text,
        "title": "Pushover Notification"
    }
    requests.post(pushover_url, data=payload)

In [13]:
push("how are you doing?")

In [105]:
def record_user_details(email, name ="Name not provided", notes= "Notes not provided"):
    push(f'Recording {name} and email {email} and notes {notes}')
    return {'recorded':"ok"}


In [106]:
def record_unknown_question(question):
    push(f'Recording question {question}')
    return {'recorded':"ok"}


In [107]:
# create a tools json file


record_user_details_json = {
    'name': 'record_user_details',
    'description': "use this tool to record user details",
    'parameter' :{
        'type':'object',
        'properties':{
            'email':{
                'type':'string',
                'description' : "The email address of this user"
            },
            'name':{
                'type':'string',
                'description' :"The user's name, if they provided it"

            },
            'notes':{
                'type':'string',
                'description' : "Any additional information about the conversation that's worth recording to give context"
            },

        },
        'required':['email'],
        "additionalProperties":False
    }



}

In [108]:
record_unknown_question_json ={
    'name':'record_unknown_question',
    'description':"Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    'parameter':{
        'type':'object',
        'parameters':{
            'question':{
                'type':'string',
                'description':"he question that couldn't be answered"
            },
        },
        'required': ['question'],
        "additionalProperties":False

    }



}

In [109]:
tools = [{'type':"function", "function":record_user_details_json}, {'type':"function", "function":record_unknown_question_json}]

In [37]:
tools

[{'type': 'function',
  'function': {'name': 'record_user_details',
   'description': 'use this tool to record user details',
   'parameter': {'type': 'object',
    'properties': {'email': {'type': 'string',
      'description': 'The email address of this user'},
     'name': {'type': 'string',
      'description': "The user's name, if they provided it"},
     'notes': {'type': 'string',
      'description': "Any additional information about the conversation that's worth recording to give context"}},
    'required': ['email'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_question',
   'description': "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
   'parameter': {'type': 'object',
    'parameters': {'question': {'type': 'string',
      'description': "he question that couldn't be answered"}},
    'required': ['question'],
    'additionalProperties': False}}}]

In [38]:
tools[0]

{'type': 'function',
 'function': {'name': 'record_user_details',
  'description': 'use this tool to record user details',
  'parameter': {'type': 'object',
   'properties': {'email': {'type': 'string',
     'description': 'The email address of this user'},
    'name': {'type': 'string',
     'description': "The user's name, if they provided it"},
    'notes': {'type': 'string',
     'description': "Any additional information about the conversation that's worth recording to give context"}},
   'required': ['email'],
   'additionalProperties': False}}}

In [110]:
# This function can take a list of tool calls, and run them. This is the IF statement!!


def handle_tool_call(tool_calls):
    results = []

    for tool_call in tool_Calls:
        tool_name  = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)

        if tool_name == "record_user_details":
            results = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            results = record_question(**arguments)
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
        return results
            

In [48]:
globals()["record_unknown_question"]("this is a really hard question")

{'recorded': 'ok'}

In [111]:
# This is a more elegant way that avoids the IF statement.

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [112]:
reader = PdfReader("resume.pdf") 
resume = ""
for page in reader.pages:
    text=page.extract_text()
    if text:
        resume+=text

with open("summary.txt", "r") as f:
    summary = f.read()




name = "poorna praneesha"


In [113]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{resume}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [114]:
def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]

    messages = [{'role':'system', 'content':system_prompt}] + history + [{'role':'user', 'content':message}]

    done = False
    while not done:

        response = groq.chat.completions.create(model = "llama-3.3-70b-versatile", messages = messages, tools = tools)
        print(response)
        # finish_reason = response.choices[0].finish_reason
        message = response.choices[0].message

        # if finish_reason == "tool_calls":
        if hasattr(message, "tool_calls") and message.tool_calls:
            # message = response.choices[0].message
            tools_calls = message.tool_calls
            results = handle_tool_calls(tools_calls)
            messages.append(message)
            messages.extend(results)

        else:
            done = True
    return message.content
        
        
# ChatCompletion(id='chatcmpl-85fc8cb9-60c2-431c-9d0f-ca5da27baaab', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I'm doing well, thanks for asking. It's great to connect with you through my website. I hope you're finding the information about my background and experience in AI engineering helpful. Is there something specific you'd like to know or discuss? Perhaps we could talk about my work in deep learning, generative AI, or my experience as a Senior AI Engineer at Hivepath?", role='assistant', executed_tools=None, function_call=None, reasoning=None, tool_calls=None))], created=1756784791, model='llama-3.3-70b-versatile', object='chat.completion', system_fingerprint='fp_6507bcfb6f', usage=CompletionUsage(completion_tokens=77, prompt_tokens=2019, total_tokens=2096, completion_time=0.221623106, prompt_time=0.16013308, queue_time=0.04814204, total_time=0.381756186), usage_breakdown=None, x_groq={'id': 'req_01k448h3q6f4krc2zjzr8zf44k'}, service_tier='on_demand')
# Traceback (most recent call last):


In [ ]:
gr.ChatInterface(chat, type="messages").launch()

## To Depoly


1. Visit https://huggingface.co and set up an account  
2. From the Avatar menu on the top right, choose Access Tokens. Choose "Create New Token". Give it WRITE permissions - it needs to have WRITE permissions! Keep a record of your new key.  
3. In the Terminal, run: `uv tool install 'huggingface_hub[cli]'` to install the HuggingFace tool, then `hf auth login` to login at the command line with your key. Afterwards, run `hf auth whoami` to che
4. ck you're logged in  
5. Take your new token and add it to your .env file: `HF_TOKEN=hf_xxx` for the future
6. From the push_notification, enter: `uv run gradio deploy` 
7. Follow its instructions: name it "conversation_about_me", specify app.py, choose cpu-basic as the hardware, say Yes to needing to supply secrets, provide your openai api key, your pushover user and token, and say "no" to github actions.  

#### More about these secrets:

If you're confused by what's going on with these secrets: it just wants you to enter the key name and value for each of your secrets -- so you would enter:  
`OPENAI_API_KEY`  
Followed by:  
`sk-proj-...`  

And if you don't want to set secrets this way, or something goes wrong with it, it's no problem - you can change your secrets later:  
1. Log in to HuggingFace website  
2. Go to your profile screen via the Avatar menu on the top right  
3. Select the Space you deployed  
4. Click on the Settings wheel on the top right  
5. You can scroll down to change your secrets (Variables and Secrets section), delete the space, etc.

#### And now you should be deployed!

If you want to completely replace everything and start again with your keys, you may need to delete the README.md that got created in this push_notification.


For more information on deployment:

https://www.gradio.app/guides/sharing-your-app#hosting-on-hf-spaces

To delete your Space in the future:  
1. Log in to HuggingFace
2. From the Avatar menu, select your profile
3. Click on the Space itself and select the settings wheel on the top right
4. Scroll to the Delete section at the bottom
5. ALSO: delete the README file that Gradio may have created inside this 1_foundations folder (otherwise it won't ask you the questions the next time you do a gradio deploy)
